In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import sys
import holidays

import sklearn
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import os
import joblib

In [2]:
class PATH:
    TRAIN_CSV=r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\서울시 평균 기온\train.csv'
    SUBMIT=r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\서울시 평균 기온\sample_submission.csv'
    OUTPUT_DIR=r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\서울시 평균 기온\OUTPUT'

class CFG:
    SEED=40
    N_SPLITS=5
    N_ESTIMATORS=1000

In [3]:
train = pd.read_csv(PATH.TRAIN_CSV)
sample = pd.read_csv(PATH.SUBMIT)

display(train.describe())
display(train.info())

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
count,23008.000000,23008.000000,23007.000000,9150.000000,23011.000000,23007.000000,22893.000000,18149.000000,22645.000000,23011.000000
mean,17.071714,8.451960,8.619277,9.593683,65.202851,2.380993,5.858826,11.933170,48.653526,12.415419
std,10.714471,10.578285,2.907096,21.966135,14.549077,0.947595,3.816941,6.419122,31.662321,10.489515
min,-13.600000,-20.200000,1.000000,0.000000,17.900000,0.100000,0.000000,0.000000,0.000000,-16.400000
25%,7.800000,-0.300000,6.600000,0.100000,54.900000,1.700000,2.200000,7.000000,17.800000,3.400000
50%,18.900000,9.200000,8.600000,1.400000,65.500000,2.200000,6.600000,11.220000,55.700000,13.800000
75%,26.400000,17.900000,10.600000,8.500000,75.800000,2.900000,9.000000,16.620000,78.000000,21.800000
max,39.600000,30.300000,19.600000,332.800000,99.800000,7.800000,13.700000,33.480000,96.900000,33.700000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB


None

In [4]:
train.isna().sum()

일시          0
최고기온        3
최저기온        3
일교차         4
강수량     13861
평균습도        0
평균풍속        4
일조합       118
일사합      4862
일조율       366
평균기온        0
dtype: int64

In [5]:
train['일시'] = pd.to_datetime(train['일시'])

In [6]:
train = train.interpolate(method = 'linear').bfill()

In [7]:
2022-1960

62

In [8]:
train['year'] = train['일시'].dt.year
train['month'] = train['일시'].dt.month
train['week'] = train['일시'].dt.isocalendar().week
train['day'] = train['일시'].dt.day
train['day_of_week'] = train['일시'].dt.dayofweek
train['sin_month'] = np.sin(2 * np.pi * train['month'] / 12)
train['cos_month'] = np.cos(2 * np.pi * train['month'] / 12)
train['spring'] = train['month'].apply(lambda x: 1 if x in [3, 4, 5] else 0)
train['summer'] = train['month'].apply(lambda x: 1 if x in [6, 7, 8] else 0)
train['fall'] = train['month'].apply(lambda x: 1 if x in [9, 10, 11] else 0)
train['winter'] = train['month'].apply(lambda x: 1 if x in [12, 1, 2] else 0)
train['spring_sin'] = np.sin(2 * np.pi * train['spring'] / 12)
train['spring_cos'] = np.cos(2 * np.pi * train['spring'] / 12)
train['summer_sin'] = np.sin(2 * np.pi * train['summer'] / 12)
train['summer_cos'] = np.cos(2 * np.pi * train['summer'] / 12)
train['fall_sin'] = np.sin(2 * np.pi * train['fall'] / 12)
train['fall_cos'] = np.cos(2 * np.pi * train['fall'] / 12)
train['winter_sin'] = np.sin(2 * np.pi * train['winter'] / 12)
train['winter_cos'] = np.cos(2 * np.pi * train['winter'] / 12)
train['sin_year'] = np.sin(2 * np.pi * (train['year'] - train['year'].min()))
train['cos_year'] = np.cos(2 * np.pi * (train['year'] - train['year'].min()))

train

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,...,spring_sin,spring_cos,summer_sin,summer_cos,fall_sin,fall_cos,winter_sin,winter_cos,sin_year,cos_year
0,1960-01-01,2.2,-5.2,7.4,0.4,68.3,1.7,6.7,4.81,28.1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,0.000000e+00,1.0
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,4.81,28.1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,0.000000e+00,1.0
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,4.81,28.1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,0.000000e+00,1.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,4.81,28.1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,0.000000e+00,1.0
4,1960-01-05,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,4.81,28.1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,10.6,0.6,69.8,1.8,8.8,10.25,91.7,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,-2.939648e-14,1.0
23007,2022-12-28,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.86,90.6,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,-2.939648e-14,1.0
23008,2022-12-29,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.88,93.8,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,-2.939648e-14,1.0
23009,2022-12-30,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.84,82.3,...,0.0,1.0,0.0,1.0,0.0,1.0,0.5,0.866025,-2.939648e-14,1.0


In [9]:
season_dict = {
    '입춘': pd.to_datetime('1960-02-04'), '우수': pd.to_datetime('1960-02-18'), '경칩': pd.to_datetime('1960-03-05'),
    '춘분': pd.to_datetime('1960-03-20'), '청명': pd.to_datetime('1960-04-05'), '곡우': pd.to_datetime('1960-04-20'),
    '입하': pd.to_datetime('1960-05-05'), '소만': pd.to_datetime('1960-05-21'), '망종': pd.to_datetime('1960-06-06'),
    '하지': pd.to_datetime('1960-06-21'), '소서': pd.to_datetime('1960-07-07'), '대서': pd.to_datetime('1960-07-22'),
    '가을': pd.to_datetime('1960-08-07'), '입추': pd.to_datetime('1960-08-23'), '처서': pd.to_datetime('1960-09-07'),
    '백로': pd.to_datetime('1960-09-22'), '추분': pd.to_datetime('1960-10-08'), '한로': pd.to_datetime('1960-10-23'),
    '상강': pd.to_datetime('1960-11-07'), '입동': pd.to_datetime('1960-11-22'), '소설': pd.to_datetime('1960-12-07'),
    '대설': pd.to_datetime('1960-12-22'), '동지': pd.to_datetime('1961-01-05'), '소한': pd.to_datetime('1961-01-20')
}

train['절기'] = train['일시'].apply(lambda x: next((key for key, value in season_dict.items() if value.month == x.month and value.day == x.day), None))
train['절기'] = train['절기'].notnull().astype(int)

for season in season_dict.keys():
    train[season] = (train['일시'] == season_dict[season]).astype(int)

one_hot_encoded = pd.get_dummies(train['절기'])

train = pd.concat([train, one_hot_encoded], axis=1)

train.drop([0, 1], axis=1, inplace=True)

train

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,...,처서,백로,추분,한로,상강,입동,소설,대설,동지,소한
0,1960-01-01,2.2,-5.2,7.4,0.4,68.3,1.7,6.7,4.81,28.1,...,0,0,0,0,0,0,0,0,0,0
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,4.81,28.1,...,0,0,0,0,0,0,0,0,0,0
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,4.81,28.1,...,0,0,0,0,0,0,0,0,0,0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,4.81,28.1,...,0,0,0,0,0,0,0,0,0,0
4,1960-01-05,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,4.81,28.1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,10.6,0.6,69.8,1.8,8.8,10.25,91.7,...,0,0,0,0,0,0,0,0,0,0
23007,2022-12-28,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.86,90.6,...,0,0,0,0,0,0,0,0,0,0
23008,2022-12-29,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.88,93.8,...,0,0,0,0,0,0,0,0,0,0
23009,2022-12-30,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.84,82.3,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X = train[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율', 'year', 'month', 'week', 'day', 'day_of_week', 'sin_month',
       'cos_month', 'spring', 'summer', 'fall', 'winter', 'spring_sin','spring_cos', 'summer_sin', 'summer_cos', 'fall_sin', 'fall_cos','winter_sin', 'winter_cos', 
       '절기', '입춘', '우수', '경칩', '춘분', '청명', '곡우', '입하', '소만', '망종', '하지', '소서', '대서', '가을', '입추', '처서', '백로', '추분', '한로',
       '상강', '입동', '소설', '대설', '동지', '소한', 'sin_year', 'cos_year']]

y = train['평균기온']

# K-FOLD 설정
kf = KFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=CFG.SEED)

# Fold별 MAE 저장할 리스트
mae_scores = []

# OOF Pred를 저장할 배열
oof_preds = np.zeros(X.shape[0])

# Fold별 결과를 담을 디렉토리 생성
if not os.path.exists(PATH.OUTPUT_DIR):
    os.mkdir(PATH.OUTPUT_DIR)


models = [
    ('CatBoost', CatBoostRegressor(n_estimators=CFG.N_ESTIMATORS, random_state=CFG.SEED)),
    ('XGBoost', XGBRegressor(n_estimators=CFG.N_ESTIMATORS, random_state=CFG.SEED)),
    ('LightGBM', LGBMRegressor(n_estimators=CFG.N_ESTIMATORS, random_state=CFG.SEED)),
    ('RandomForest', RandomForestRegressor(n_estimators=CFG.N_ESTIMATORS, random_state=CFG.SEED)),
    ('ExtraTrees', ExtraTreesRegressor(n_estimators=CFG.N_ESTIMATORS, random_state=CFG.SEED))
]

oof_preds_ensemble = np.zeros(X.shape[0])

for fold, (train_index, valid_index) in enumerate(kf.split(X)):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    fold_preds = []

    for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        fold_preds.append(y_pred)

        joblib.dump(model, PATH.OUTPUT_DIR + f'/{name}_fold{fold}.joblib')

    fold_preds_mean = np.mean(fold_preds, axis=0)
    mae = mean_absolute_error(y_valid, fold_preds_mean)
    mae_scores.append(mae)

    oof_preds_ensemble[valid_index] = fold_preds_mean

Learning rate set to 0.064871
0:	learn: 9.8933430	total: 146ms	remaining: 2m 25s
1:	learn: 9.2899591	total: 151ms	remaining: 1m 15s
2:	learn: 8.7235036	total: 155ms	remaining: 51.5s
3:	learn: 8.1926586	total: 160ms	remaining: 39.8s
4:	learn: 7.6983639	total: 164ms	remaining: 32.6s
5:	learn: 7.2431289	total: 168ms	remaining: 27.9s
6:	learn: 6.8050002	total: 174ms	remaining: 24.6s
7:	learn: 6.4116102	total: 179ms	remaining: 22.2s
8:	learn: 6.0389863	total: 184ms	remaining: 20.2s
9:	learn: 5.6853332	total: 188ms	remaining: 18.6s
10:	learn: 5.3455610	total: 193ms	remaining: 17.4s
11:	learn: 5.0420261	total: 198ms	remaining: 16.3s
12:	learn: 4.7423575	total: 204ms	remaining: 15.5s
13:	learn: 4.4622700	total: 208ms	remaining: 14.6s
14:	learn: 4.2054681	total: 212ms	remaining: 13.9s
15:	learn: 3.9708264	total: 216ms	remaining: 13.3s
16:	learn: 3.7401182	total: 221ms	remaining: 12.8s
17:	learn: 3.5235526	total: 226ms	remaining: 12.3s
18:	learn: 3.3203645	total: 230ms	remaining: 11.9s
19:	learn

In [ ]:
average_mae = np.mean(mae_scores)
print(f'Average Mean Absolute Error: {average_mae}')

In [ ]:
sample['평균기온'] = pd.DataFrame(oof_preds_ensemble[:358], columns=['평균기온'])

sample.to_csv(PATH.OUTPUT_DIR + '/0101-1.csv', index=False)

sample

In [ ]:
# 파일 경로
train_path = r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\서울시 평균 기온\train.csv' # 학습 데이터 파일 경로
submission_path = r'C:\Users\dlwks\OneDrive\바탕 화면\VSCode\서울시 평균 기온\sample_submission.csv' # 제출 양식 파일 경로

# 데이터 불러오기
train_df = pd.read_csv(train_path)
submission_df = pd.read_csv(submission_path)

display(train_df.head())
display(submission_df.head())

In [ ]:
train_df['일시'] = pd.to_datetime(train_df['일시'])
train_df = train_df.set_index('일시')

# 데이터의 시간 간격 지정
train_df.index.freq = 'D'

# 일시 컬럼이 인덱스로 할당됩니다.  
train_df.head()

In [ ]:
# prophet에서 데이터를 인식하도록 일시는 ds로, target값인 평균기온은 y로 지정해줍니다.
train_df = train_df.reset_index()
train_df = train_df.rename(columns={'일시': 'ds', '평균기온': 'y'})

In [ ]:
#모델 학습

from prophet import Prophet

prophet = Prophet()
prophet.fit(train_df)

In [ ]:
#모델 예측
future_data = prophet.make_future_dataframe(periods = 358, freq = 'd') #periods는 예측할 기간
forecast_data = prophet.predict(future_data)
forecast_data[['ds','yhat']].tail(5)

In [ ]:
submission_df['평균기온'] = forecast_data.yhat[-358:].values
submission_df